# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [18]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.core.experiment import Experiment
from azureml.core.model import Model
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.utilities import get_primary_metrics
from azureml.widgets import RunDetails
import joblib

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

I am using the Mushroom Classification dataset from Kaggle (https://www.kaggle.com/uciml/mushroom-classification) in order to predict whether certain mushrooms are safe to eat or are deadly poisonous. Besides the binary target column "class", which contains the values e (edible) and p (poisonous), the dataset contains various feature columns with geographical, physical and biological characteristics of the mushrooms. This dataset was originally contributed to the UCI Machine Learning repository nearly 30 years ago but has seen increasing use as mushroom hunting (otherwise known as "shrooming") is enjoying new peaks in popularity.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# Create an experiment
experiment_name = 'mushrooms-automl'
experiment=Experiment(ws, experiment_name)

# Retrieve the data from the workspace
dataset = Dataset.get_by_name(ws, name='mushrooms')
df = dataset.to_pandas_dataframe()

# Show the first rows
df.head()

## Compute Target

In [ ]:
# Retrieve compute target
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.
- "max_concurrent_iterations" is set to 4 as this number should be smaller than the node count of the cluster (which is 5).
- AUC_weighted is used as "primary metric". The dataset is almost balanced so accuracy could have been used as well. However, the poison class is in a slighty minority. With the AUC_weighted metric we can reflect the fact that it is more important to correctly identify poisonous mushrooms (the minority class) than it is to correctly identify edible mushrooms since eating a poisonous mushroom could result in death.

In [ ]:
# Check available primary metrics for classification
get_primary_metrics("classification")

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 60,
    "max_concurrent_iterations": 5,
    "primary_metric" : "AUC_weighted"
}

project_folder = "./automl"

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target,
                             task = "classification",
                             training_data = dataset,
                             label_column_name = "class",   
                             path = project_folder,
                             enable_early_stopping=True,
                             featurization = "auto",
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_automl_run, automl_model = remote_run.get_output()
print(best_automl_run)
print(automl_model)

In [ ]:
#TODO: Save the best model
joblib.dump(automl_model, "models/automl_model.pkl")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service